# README

# SimSort
## *Using dimensionality reduction and graphs to group similar texts*

---

## Purpose

This project provides several implementations (command line accessible python script, configurable notebook, and api) of a simple sort function that groups semantically similar texts together. This algorithm uses some common text embedding and clustering techniques to arrange texts by similarity on a single dimension, resulting in a sorted list of texts much like sorting alphabetically or by text length. Simsort specifically targets instances where it is useful to group similar texts together (as in traditional higher dimensional clustering), but the groups should also be somehow arranged in relation to one another (i.e. sorted in some interpretable way).

---

## Approach
The simsort algorithm takes a series of unordered texts and groups them such that similar texts are together in the resulting output. Traditional sort methods are based on some absolute metric (alphabetical order, number of tokens/characters, presence of given token/character), and have predictable endpoints based on the sort criterion (a -> z, low numbers -> high numbers, short texts -> long texts). In this way, we can think of sorting techniques as placing all data observations on a 1-dimensional line, with the position of an observation corresponding to its relative similarity to one of the two endpoints of the line.

Using this framework, simsort defines the endpoints of the sorting axis as the two most dissimlar observations, with every other observation existing somewhere in between the two endpoints. Similarity is defined using the distance between the embedding vectors for each text, and these embedding vectors are projected onto a 1-dimensional space via 1) dimensionality reduction or 2) solving for the shortest path visiting all nodes of a graph based on distances between embeddings.

Simsort seeks to provide a more powerful and hopefully useful sort utility for working with text data, especially in graphical tools like spreadsheets for manual data analysis.

### Algorithm
Steps for the simsort algorithm are as follows:
- Embed unsorted texts using model of choice
- Pass embedding array into either:
  - A dimensionality reduction algorithm, bringing the dimensionality of the embeddings to n = 1 and
  - A weighted graph of pairwise distances between embedding vectors, using a shortest path algorithm to find the shortest path that visits all nodes in the graph
- Sort the original texts by either the sorted embedding values or shortest path indices

### Embedding Models
Most embedding approaches will work in the simsort algorithm, but the present implementation uses the following models:
- [Universal Sentence Encoder](https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder)
- [Term Frequency-Inverse Document Frequency](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Sentence Transformers all-MiniLM-L6-v1](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v1)

### Order Solvers
The field of applicable dimensionality reduction techniques/shortest path algorithms is more limited, and most are used in the present project:
- [Locally Linear Embedding](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html)
- [MultiDimensional Scaling](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)
- [Principal Component Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
- [Isomap](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html)
- [Traveling Salesman Shortest Path, Christofides Approximation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.christofides.html)
- [Traveling Salesman Shortest Path, Greedy Approximation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.greedy_tsp.html)

# Imports & Utils



In [1]:
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import networkx as nx
from sklearn.manifold import LocallyLinearEmbedding, Isomap, MDS
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sentence_transformers import SentenceTransformer
import os
import json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow_hub as hub

In [3]:
algo_key = {
	"lle" : {"solver" : LocallyLinearEmbedding, "type" : "sklearn"},
	"isomap" : {"solver" : Isomap, "type" : "sklearn"},
	"mds" : {"solver" : MDS, "type" : "sklearn"},
	"pca" : {"solver" : PCA, "type" : "sklearn"},
	"tsp_c" : {"solver" : nx.approximation.christofides, "type" : "networkx"},
	"tsp_g" : {"solver" : nx.approximation.greedy_tsp, "type" : "networkx"},
	}

In [4]:
embedder_key = {
    "use" : hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2"),
    "tfidf" : TfidfVectorizer(),
    "minilm" : SentenceTransformer("all-MiniLM-L6-v1"),
}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.95k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def embed_texts(texts: list, embedder: str) -> np.ndarray:
  """
	Embed texts with specified embedding model

	Args:
		texts (list): Texts to embed
		embedder (str): Embedding model

	Returns:
		np.ndarray: Embedding vectors for texts
	"""
  model = embedder_key[embedder]

  if embedder == "use":
    out = model(texts)

  if embedder == "tfidf":
    out = model.fit_transform(texts).toarray()

  if embedder == "minilm":
    out = model.encode(texts)

  return out

def create_graph_from_embeddings(embeddings: np.ndarray) -> nx.Graph:
	"""
	Create weighted graph of distances between embedded texts, where texts correspond to nodes and pairwise distances are used as edge weights

	Args:
		embeddings (np.ndarray): Embedding vectors for texts

	Returns:
		nx.Graph: Weighted pairwise distance graph
	"""
	distances = pairwise_distances(embeddings)
	pair_indices = np.ndindex(distances.shape)
	distance_values = distances.flatten()
	edges = [(inds[0], inds[1], dist) for inds, dist in zip(pair_indices, distance_values)]

	out = nx.Graph()
	out.add_weighted_edges_from(edges)

	return out

def calculate_sklearn_indices(embeddings: np.ndarray, dimension_reducer) -> list:
	"""
	Get sort indices using sklearn dimension reducer

	Args:
		embeddings (np.ndarray): Embedding vectors for texts
		dimension_reducer: sklearn.manifold embedder, used to reduce the dimensionality of provided embedding vectors to 1

	Returns:
		list: Sort indices for embedding array
	"""
	reduced_embeddings = dimension_reducer(n_components = 1).fit_transform(embeddings).flatten()
	out = np.argsort(reduced_embeddings).tolist()
	return out

def calculate_networkx_indices(graph: nx.Graph, path_approximator) -> list:
	"""
	Get sort indices using networkx shortest path approximator

	Args:
		graph (nx.Graph): Graph representation of pairwise distances between embedded texts
		path_approximator: networkx.approximation algorithm, used to calculate the shortest path between all nodes of the graph (see: traveling salesman problem)

	Returns:
		list: Sort indices for distance graph
	"""
	return nx.approximation.traveling_salesman_problem(G = graph, cycle = False, method = path_approximator)

def single_simsort(args: dict) -> dict:
	"""
	Run a single iteration of the simsort algorithm in simsort.py, using provided arguments (parsed from command line flags)

	Args:
		args (dict): User inputs

	Returns:
		dict: Results of simsort algorithm, including original texts, sort indices, and sorted texts
	"""
	algo = algo_key[args["solver"]]

	with open(args["filename"], "r") as f:
		texts = json.load(f)["texts"]
		out = {"texts" : texts}
		print(f"Imported texts from {args['filename']}")

	print(f"Sorting texts with embedder '{args['embedder']}' and solver '{args['solver']}'")

	embs = embed_texts(texts, args['embedder'])

	if algo["type"] == "sklearn":
		index = calculate_sklearn_indices(embs, algo["solver"])
		out["index"] = index
		out["sorted"] = [texts[i] for i in index]

	if algo["type"] == "networkx":
		graph = create_graph_from_embeddings(embs)
		index = calculate_networkx_indices(graph, algo["solver"])
		out["index"] = index
		out["sorted"] = [texts[i] for i in index]

	if args['outfile']:
		with open(args['outfile'], "w") as f:
			f.write(json.dumps(out))

	print(f"Saving results to {args['outfile']}")
	print(out)
	return out

# User Inputs

In [6]:
#@markdown ## Filename:
#@markdown json file to read texts from\
#@markdown Assumes texts are stored in json array with field name 'texts', i.e.: {'texts' : ['text1', 'text2', ...]}
filename = "texts.json" #@param

#@markdown ## Outfile:
#@markdown json file to write sorted texts and indices to\
#@markdown Output has structure {'texts': ['text1', 'text2', ...], 'index': [index1, index2, ...], 'sorted': ['sorted_text1', 'sorted_text2', ...]}
outfile = "output.json" #@param

#@markdown ## Embedder:
#@markdown Embedding model to apply to texts\
#@markdown One of:
#@markdown - use [Universal Sentence Encoder](https://www.tensorflow.org/hub/tutorials/semantic_similarity_with_tf_hub_universal_encoder)
#@markdown - tfidf [Term Frequency-Inverse Document Frequency](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
#@markdown - minilm [Sentence Transformers all-MiniLM-L6-v1](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v1)
embedder = "use" #@param ["use", "tfidf", "minilm"]

#@markdown ## Solver:
#@markdown Solver (dimension reduction or shortest path algorithm) to use when calculating sort order\
#@markdown One of:
#@markdown - lle [Locally Linear Embedding](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.LocallyLinearEmbedding.html)
#@markdown - mds [MultiDimensional Scaling](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html)
#@markdown - pca [Principal Component Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)
#@markdown - isomap [Isomap](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.Isomap.html)
#@markdown - tsp_c [Traveling Salesman Shortest Path, Christofides Approximation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.christofides.html)
#@markdown - tsp_g [Traveling Salesman Shortest Path, Greedy Approximation](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.greedy_tsp.html)
solver = "tsp_c" #@param ["mds", "lle", "isomap", "tsp_c", "tsp_g", "pca"]

args = {
    "filename" : filename,
    "outfile" : outfile,
    "embedder" : embedder,
    "solver" : solver
    }

# Sorting & Output

In [8]:
single_simsort(args)

Imported texts from texts.json
Sorting texts with embedder 'use' and solver 'tsp_c'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [1, 0, 11, 14, 12, 13, 9, 10, 5, 6, 8, 7, 2, 4, 3], 'sorted': ["i'm shocked", 'oh wow', 'def', '0', '123', '555', 'abc', 'cde', 'chicken nugget', 'ham burger', 'salad', 'french fries', 'cheese is a food', 'dogs drool', 'cats rule']}


{'texts': ['oh wow',
  "i'm shocked",
  'cheese is a food',
  'cats rule',
  'dogs drool',
  'chicken nugget',
  'ham burger',
  'french fries',
  'salad',
  'abc',
  'cde',
  'def',
  '123',
  '555',
  '0'],
 'index': [1, 0, 11, 14, 12, 13, 9, 10, 5, 6, 8, 7, 2, 4, 3],
 'sorted': ["i'm shocked",
  'oh wow',
  'def',
  '0',
  '123',
  '555',
  'abc',
  'cde',
  'chicken nugget',
  'ham burger',
  'salad',
  'french fries',
  'cheese is a food',
  'dogs drool',
  'cats rule']}

# Test Mode
Run script in test mode, evaluating all combinations of embeddings and solvers on the provided texts to see which produces the best results

In [9]:
	test = {}

	for embedder in ["use", "tfidf", "minilm"]:
		for solver in ["lle", "isomap", "mds", "pca", "tsp_c", "tsp_g"]:
			name = f"{solver}.{embedder}"
			args['solver'] = solver
			args['embedder'] = embedder
			res = single_simsort(args)
			test[name] = res

	with open("test_results.json", "w") as f:
		f.write(json.dumps(test))

Imported texts from texts.json
Sorting texts with embedder 'use' and solver 'lle'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [4, 3, 2, 6, 7, 8, 5, 10, 9, 1, 0, 12, 11, 13, 14], 'sorted': ['dogs drool', 'cats rule', 'cheese is a food', 'ham burger', 'french fries', 'salad', 'chicken nugget', 'cde', 'abc', "i'm shocked", 'oh wow', '123', 'def', '555', '0']}
Imported texts from texts.json
Sorting texts with embedder 'use' and solver 'isomap'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [2, 4, 6, 8, 7, 3, 5, 10, 1, 9, 0, 12, 11, 14, 13], 'sorted': ['cheese is a food', 'dogs drool', 'ham burger', 'salad', 'french fries', 'cats rule', 'chicken nu

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [10, 13, 11, 12, 0, 3, 4, 14, 1, 2, 8, 9, 5, 6, 7], 'sorted': ['cde', '555', 'def', '123', 'oh wow', 'cats rule', 'dogs drool', '0', "i'm shocked", 'cheese is a food', 'salad', 'abc', 'chicken nugget', 'ham burger', 'french fries']}
Imported texts from texts.json
Sorting texts with embedder 'tfidf' and solver 'mds'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [6, 12, 13, 1, 10, 9, 8, 14, 11, 2, 3, 7, 4, 0, 5], 'sorted': ['ham burger', '123', '555', "i'm shocked", 'cde', 'abc', 'salad', '0', 'def', 'cheese is a food', 'cats rule', 'french fries', 'dogs drool', 'oh wow', 'chicken nugget']}
Imported t

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [6, 2, 5, 7, 8, 4, 3, 1, 9, 10, 11, 13, 0, 12, 14], 'sorted': ['ham burger', 'cheese is a food', 'chicken nugget', 'french fries', 'salad', 'dogs drool', 'cats rule', "i'm shocked", 'abc', 'cde', 'def', '555', 'oh wow', '123', '0']}
Imported texts from texts.json
Sorting texts with embedder 'minilm' and solver 'isomap'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [14, 0, 11, 12, 13, 10, 1, 9, 3, 4, 8, 5, 7, 6, 2], 'sorted': ['0', 'oh wow', 'def', '123', '555', 'cde', "i'm shocked", 'abc', 'cats rule', 'dogs drool', 'salad', 'chicken nugget', 'french fries', 'ham burger', 'cheese is a food']}
Import

/usr/local/lib/python3.10/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [6, 14, 4, 9, 12, 13, 8, 0, 7, 10, 2, 11, 5, 3, 1], 'sorted': ['ham burger', '0', 'dogs drool', 'abc', '123', '555', 'salad', 'oh wow', 'french fries', 'cde', 'cheese is a food', 'def', 'chicken nugget', 'cats rule', "i'm shocked"]}
Imported texts from texts.json
Sorting texts with embedder 'minilm' and solver 'pca'
Saving results to output.json
{'texts': ['oh wow', "i'm shocked", 'cheese is a food', 'cats rule', 'dogs drool', 'chicken nugget', 'ham burger', 'french fries', 'salad', 'abc', 'cde', 'def', '123', '555', '0'], 'index': [12, 14, 13, 0, 11, 9, 10, 1, 3, 4, 8, 5, 6, 7, 2], 'sorted': ['123', '0', '555', 'oh wow', 'def', 'abc', 'cde', "i'm shocked", 'cats rule', 'dogs drool', 'salad', 'chicken nugget', 'ham burger', 'french fries', 'cheese is a food']}
Imported 